In [1]:
!pip install opencv-python opencv-python-headless
!pip --version

pip 24.0 from /home/qphulong/workspace/stella_slam_camera_rotation/venv/lib/python3.12/site-packages/pip (python 3.12)


In [35]:
import subprocess
def run_shell_command(command):
    """Helper function to run shell commands and handle errors."""
    result = subprocess.run(command, capture_output=True, text=True, shell=True)
    if result.returncode != 0:
        print(f"Error: {result.stderr}")
    return result.stdout

In [ ]:
# 1. PUT THE VIDEO IN ./input MANUALLY
# 2. RUN manual_mask.py to manual mask the config (color the part where slam shoudl ignore)

In [3]:
# run this command to start the container
command = "docker run -it --rm --privileged -e DISPLAY=$DISPLAY -v /tmp/.X11-unix/:/tmp/.X11-unix:ro stella_vslam-iridescense"

In [36]:
# get the container id
import subprocess
print("Getting the first Docker container ID...")
container_id = subprocess.check_output("docker ps -q", shell=True).decode().strip()
print(f"Container ID: {container_id}")

Getting the first Docker container ID...
Container ID: 6572a64853a3


In [37]:
# get video name
import os

# Path to the input directory
input_dir = './input/'

# List all files in the directory
files = [f for f in os.listdir(input_dir) if f.endswith('.mp4')]

# Check if there is exactly one .mp4 file
if len(files) == 1:
    # Store the name of the .mp4 file
    video_name = files[0]
    print(f"Video name: {video_name}")
else:
    # Raise an error if the condition is not met
    raise ValueError("The directory must contain exactly one .mp4 file.")


Video name: o.mp4


In [38]:
import ffmpeg

# Function to extract video metadata using ffmpeg
def extract_video_metadata(video_path):
    # Use ffmpeg.probe to retrieve metadata of the video
    probe = ffmpeg._probe.probe(video_path, v='error', select_streams='v:0', show_entries='stream=width,height,r_frame_rate')
    
    # Extract relevant metadata from the probe results
    video_stream = probe['streams'][0]
    
    # Extract FPS, width (cols), and height (rows)
    fps = eval(video_stream['r_frame_rate'])  # 'r_frame_rate' is in the form of "numerator/denominator"
    cols = video_stream['width']
    rows = video_stream['height']
    
    return {
        'fps': fps,
        'cols': cols,
        'rows': rows
    }
# Extract metadata
video_metadata = extract_video_metadata('./input/'+video_name)
print(video_metadata)


{'fps': 29.97, 'cols': 1920, 'rows': 1080}


In [39]:
import yaml

yaml_content = f"""
Camera:
  name: "RICOH THETA S 960"
  setup: "monocular"
  model: "equirectangular"
  fps: {video_metadata['fps']}
  cols: {video_metadata['cols']}
  rows: {video_metadata['rows']}
  color_order: "RGB"

Preprocessing:
  min_size: 800
  mask_rectangles:
  


Feature:
  name: "default ORB feature extraction setting"
  scale_factor: 1.2
  num_levels: 8
  ini_fast_threshold: 20
  min_fast_threshold: 7

Mapping:
  backend: "g2o"
  baseline_dist_thr_ratio: 0.02
  redundant_obs_ratio_thr: 0.95
  num_covisibilities_for_landmark_generation: 20
  num_covisibilities_for_landmark_fusion: 20
  residual_deg_thr: 0.4

Tracking:
  backend: "g2o"

LoopDetector:
  backend: "g2o"
  enabled: true
  reject_by_graph_distance: true
  min_distance_on_graph: 50

GraphOptimizer:
  min_num_shared_lms: 200

GlobalOptimizer:
  thr_neighbor_keyframes: 100

System:
  map_format: "msgpack"
  num_grid_cols: 96
  num_grid_rows: 48
"""

# Write the content to a file
file_path = './input/config.yaml'

with open(file_path, 'w') as file:
    file.write(yaml_content)

print(f"YAML file has been created at {file_path}")


YAML file has been created at ./input/config.yaml


In [40]:
# copy video and config to container
print(f"Copying inputs to container {container_id}...")
subprocess.run(
    f"docker cp ./input {container_id}:/stella_vslam_examples/build/input",
    shell=True
)

Copying inputs to container 6572a64853a3...


CompletedProcess(args='docker cp ./input 6572a64853a3:/stella_vslam_examples/build/input', returncode=0)

In [41]:
# repair the container
subprocess.run(
    f"docker exec {container_id} mkdir output",
    shell=True
)
subprocess.run(
    f'docker exec {container_id} curl -sL "https://github.com/stella-cv/FBoW_orb_vocab/raw/main/orb_vocab.fbow" -o orb_vocab.fbow',
    shell=True
)

CompletedProcess(args='docker exec 6572a64853a3 curl -sL "https://github.com/stella-cv/FBoW_orb_vocab/raw/main/orb_vocab.fbow" -o orb_vocab.fbow', returncode=0)

In [42]:
# run the container
subprocess.run(
    f"docker exec {container_id} ./run_video_slam \
    -v ./orb_vocab.fbow \
    -c ./input/config.yaml \
    -m ./input/{video_name} \
    --map-db-out map.msg \
    --no-sleep \
    --eval-log-dir output \
    --start-timestamp 0 \
    --mask ./input/mask.png",
    shell=True
)

[2025-01-16 10:35:34.128] [I] config file loaded: ./input/config.yaml
[2025-01-16 10:35:34.128] [I] 
original version of OpenVSLAM,
Copyright (C) 2019,
National Institute of Advanced Industrial Science and Technology (AIST)
All rights reserved.
stella_vslam (the changes after forking from OpenVSLAM),
Copyright (C) 2022, stella-cv, All rights reserved.

This is free software,
and you are welcome to redistribute it under certain conditions.
See the LICENSE file.

Camera:
  name: RICOH THETA S 960
  setup: monocular
  model: equirectangular
  fps: 29.97
  cols: 1920
  rows: 1080
  color_order: RGB
Preprocessing:
  min_size: 800
  mask_rectangles: ~
Feature:
  name: default ORB feature extraction setting
  scale_factor: 1.2
  num_levels: 8
  ini_fast_threshold: 20
  min_fast_threshold: 7
Mapping:
  backend: g2o
  baseline_dist_thr_ratio: 0.02
  redundant_obs_ratio_thr: 0.95
  num_covisibilities_for_landmark_generation: 20
  num_covisibilities_for_landmark_fusion: 20
  residual_deg_thr: 0.4

CompletedProcess(args='docker exec 6572a64853a3 ./run_video_slam     -v ./orb_vocab.fbow     -c ./input/config.yaml     -m ./input/o.mp4     --map-db-out map.msg     --no-sleep     --eval-log-dir output     --start-timestamp 0     --mask ./input/mask.png', returncode=0)

In [43]:
subprocess.run(
    f"docker cp {container_id}:/stella_vslam_examples/build/output .",
    shell=True
)

CompletedProcess(args='docker cp 6572a64853a3:/stella_vslam_examples/build/output .', returncode=0)

In [ ]:
# visualize animation
# run animation.py

# overlay the path to video
# run overlay.py (overlay currently does not work well for y)
# tam thoi cu xai overlay.py chu cung ko chac cai code overlay no dung :), co the no sai a.